In [26]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from aicssegmentation.core.utils import get_middle_frame, hole_filling, get_3dseed_from_mid_frame
from skimage.morphology import remove_small_objects, watershed, dilation, ball

from tifffile import imsave

In [127]:
filename = r"\\allen\aics\assay-dev\users\Sandi\nuc-morph-analysis\speckle-analysis\for-presentation\raw\20230425_L02-01_processed_C=0_T=400.tif"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


(1, 1, 148, 2047, 848)


In [128]:
#####################
structure_channel = 0
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
#view(single_fluorescent_view(struct_img0))

In [129]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [4000]
gaussian_smoothing_sigma = 1
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with 2d gaussian filter slice by slice 
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

intensity norm: min-max norm with upper bound 4000


In [130]:
structure_img_smooth.shape

(148, 2047, 848)

In [131]:
mid_z = get_middle_frame(structure_img_smooth, method='intensity') # 50 

In [132]:
f2_param = [[5, 0.71]] # original:[[1, 0.01],[2, 0.01],[3, 0.01]]
bw_mid_z = filament_2d_wrapper(structure_img_smooth[mid_z,:,:], f2_param)
#view(segmentation_quick_view(bw_mid_z))

In [133]:
# 2D filament filter

f2_param = [[4, 0.91]] # original:[[1, 0.01],[2, 0.01],[3, 0.01]] current: [5,0.71]
filament = filament_2d_wrapper(structure_img_smooth, f2_param)

In [134]:
filament.shape

(148, 2047, 848)

In [135]:
# remove small objects

minArea = 1000 
seg = remove_small_objects(filament>0, min_size=minArea, connectivity=1, in_place=False) #removed spots 

In [136]:
# save image

seg = seg >0
out=seg.astype(np.uint8)
out[out>0]=255
imsave("20230425_L02-01_SEGlaminshell_C=0_T=400.tif", out)

In [203]:
#view(segmentation_quick_view(bw_mid_z))

In [138]:
# # applying watershed

# hole_max = 40000
# hole_min = 400


# bw_fill_mid_z = hole_filling(bw_mid_z, hole_min, hole_max) # filled in nucleus
# seed = get_3dseed_from_mid_frame(np.logical_xor(bw_fill_mid_z, bw_mid_z), struct_img.shape, mid_z, hole_min)
# bw_filled = watershed(struct_img, seed.astype(int), watershed_line=True)>0 # outline only

# # get the shell
# seg = np.logical_xor(bw_filled, dilation(bw_filled, selem=ball(1)))

In [139]:
# viewer_bw = view(segmentation_quick_view(seg))
# viewer_bw